# Explainability Metrics


### Algorithm Class Score 

In [12]:
from difflib import get_close_matches
import tensorflow as tf
import torch.nn as nn

def algorithm_class_score(clf):

    alg_score = {
    "RandomForestClassifier": 4,
    "KNeighborsClassifier": 3,
    "SVC": 2,
    "GaussianProcessClassifier": 3,
    "DecisionTreeClassifier": 5,
    "MLPClassifier": 1,
    "AdaBoostClassifier": 3,
    "GaussianNB": 3.5,
    "QuadraticDiscriminantAnalysis": 3,
    "LogisticRegression": 4,
    "LinearRegression": 3.5,
    }

    clf_name = type(clf).__name__

    # Check if the clf_name is in the dictionary
    if clf_name in alg_score:
        exp_score = alg_score[clf_name]
        return exp_score 

    # Check if the model is a Neural Network
    if isinstance(clf, tf.keras.Model) or isinstance(clf, tf.Module) or isinstance(clf, nn.Module):
        return 1
    
    # If not, try to find a close match
    close_matches = get_close_matches(clf_name, alg_score.keys(), n=1, cutoff=0.6)
    if close_matches:
        exp_score = alg_score[close_matches[0]]
        return exp_score
    
    # If no close match found 
    print(f"No matching score found for '{clf_name}'")
    return None

In [2]:
# Example Decision Tree Classifer and Regressor
from sklearn import tree

Classifier = tree.DecisionTreeClassifier()
Regressor = tree.DecisionTreeRegressor()

print(type(Classifier).__name__)
print(algorithm_class_score(Classifier))

print(type(Regressor).__name__)
print(algorithm_class_score(Regressor))

DecisionTreeClassifier
5
DecisionTreeRegressor
5


In [3]:
# Example Neural Network Tensorflow 
import tensorflow as tf

TFNN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

print(type(TFNN).__name__)
print(algorithm_class_score(TFNN))

Sequential
1


c:\Users\anton\anaconda3\envs\INV\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Custom non-sequential NN using keras
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(5, activation="softmax")
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dropout(x, training=training)
        return self.dense2(x)

model = MyModel()

print(type(model).__name__)
print(algorithm_class_score(model))

MyModel
1


In [4]:
# Example Neural Network Pytoch
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(128, 64)  
        self.fc2 = nn.Linear(64, 32)  
        self.fc3 = nn.Linear(32, 1)    

    def forward(self, x):
        x = torch.relu(self.fc1(x))   
        x = torch.relu(self.fc2(x))    
        x = torch.sigmoid(self.fc3(x)) 
        return x
    
TOCHNN = NeuralNetwork()

print(type(TOCHNN).__name__)
print(algorithm_class_score(TOCHNN))

NeuralNetwork
1


### Feature Correlation Score

In [3]:
import numpy as np
import pandas as pd

# The higher the score, the smaller the percentage of features with hight coorelation in relation to the average coorelation 
def correlated_features_score(train_data, test_data, thresholds=[0.05, 0.16, 0.28, 0.4], target_column=None, verbose=False):
    
    test_data = test_data.copy()
    train_data = train_data.copy()
     
    if target_column:
        X_test = test_data.drop(target_column, axis=1)
        X_train = train_data.drop(target_column, axis=1)
    else:
        X_test = test_data.iloc[:,:-1]
        X_train = train_data.iloc[:,:-1]
        
    
    df_comb = pd.concat([X_test, X_train])
    df_comb = df_comb._get_numeric_data()
    corr_matrix = df_comb.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Compute average and standar deviation from upper correlation matrix 
    avg_corr = upper.values[np.triu_indices_from(upper.values,1)].mean()
    std_corr = upper.values[np.triu_indices_from(upper.values,1)].std()

    # Find features with correlation greater than avg_corr + std_corr
    to_drop = [column for column in upper.columns if any(upper[column] > (avg_corr+std_corr))]
    if verbose: print(f"Removed features: {to_drop}")
    
    pct_drop = len(to_drop)/len(df_comb.columns)
    
    bins = thresholds
    score = 5-np.digitize(pct_drop, bins, right=True) 
    
    return score

For experimental purposes there will be used the following datasets:

- [Healthcare Diabetes Dataset](https://www.kaggle.com/datasets/nanditapore/healthcare-diabetes)
- [Iris Dataset](https://www.kaggle.com/datasets/uciml/iris)

In [4]:
# Example with Healthcare Diabetes Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

health = pd.read_csv('Data/Healthcare-Diabetes.csv')

health_X = health[health.columns[1:9]]
health_y = health[health.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(health_X, health_y, test_size=0.33, random_state=42)

print(correlated_features_score(X_train, X_test, verbose=True))

Removed features: ['Insulin', 'BMI']
2


C:\Users\anton\AppData\Local\Temp\ipykernel_9748\3473009596.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [5]:
# Example with Iris Dataset
iris = pd.read_csv('Data/iris.csv')

iris_X = iris[iris.columns[:5]]
iris_y = iris['class']

X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)
print(correlated_features_score(X_train, X_test, verbose=True))


Removed features: ['petallength']
3


C:\Users\anton\AppData\Local\Temp\ipykernel_9748\3473009596.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


### Model Size Score

In [6]:
import numpy as np

# Returns a score based on the number of attributes(columns) in the dataset
def model_size_score(test_data, thresholds = np.array([10,30,100,500])):
    print(test_data.shape)
    dist_score = 5- np.digitize(test_data.shape[1]-1 , thresholds, right=True) # -1 for the id?
    
    return dist_score

print(model_size_score(iris_X))

(150, 5)
5


### Feature Relevance Score

In [106]:
def feature_relevance_score(clf):

    scale_factor = 1.5 
    distri_threshold = 0.5

    # Feature Importance for Regressions 
    if (type(clf).__name__ == 'LogisticRegression') or (type(clf).__name__ == 'LinearRegression'): 
        importance = clf.coef_.flatten()

        total = 0
        for i in range(len(importance)):
            total += abs(importance[i])

        for i in range(len(importance)):
            importance[i] = abs(importance[i]) / total

    # Feature Importance fo Random Forest, model needs to be fitted
    elif  (type(clf).__name__ == 'RandomForestClassifier') or (type(clf).__name__ == 'DecisionTreeClassifier'):
        importance = clf.feature_importances_
   
    else:
        return None
    

    # absolut values
    importance = importance
    indices = np.argsort(importance)[::-1] # indice of the biggest value in the importance list
    importance = importance[indices]
    
    # calculate quantiles for outlier detection
    q1, q3 = np.percentile(importance, [25,75])
    lower_threshold , upper_threshold = q1 - scale_factor*(q3-q1),  q3 + scale_factor*(q3-q1) 
    
    # percentage of features that concentrate distri_threshold percent of all importance
    pct_dist = sum(np.cumsum(importance) < distri_threshold) / len(importance)
    
    return pct_dist
    

'''close_matches = get_close_matches(clf_name, alg_score.keys(), n=1, cutoff=0.6)
    if close_matches:
        exp_score = alg_score[close_matches[0]]
        return exp_score'''
# add flexibility in the model


'close_matches = get_close_matches(clf_name, alg_score.keys(), n=1, cutoff=0.6)\n    if close_matches:\n        exp_score = alg_score[close_matches[0]]\n        return exp_score'

In [76]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

for i in range(2,11,2):
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=i, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=2, random_state=42)
    clf = RandomForestClassifier(random_state=123)
    clf.fit(X,y)

    print(feature_relevance_score(clf))

0.0
0.2
0.2
0.3
0.3


In [107]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

for i in range(2,11,2):
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=i, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=2, random_state=42)
    clf = LogisticRegression()
    clf.fit(X,y)
    
    print(feature_relevance_score(clf))

0.0
0.1
0.1
0.2
0.2


In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

for i in range(2,11,2):
    X, y = make_regression(n_samples=1000, n_features=10, n_informative=i, n_targets=1, random_state=123)
    clf = LinearRegression()
    clf.fit(X,y)
    
    print(feature_relevance_score(clf))


0.0
0.1
0.1
0.2
0.3


### Feature Importance

In [ ]:
def get_feature_importance_cv(test_sample, model, cfg):
    """Calculates feature importance coefficient of variation
       :param test_sample: one test sample
       :param model: the model
       :param cfg: configs
       :return: the coefficient of variation of the feature importance scores, [0, 1]
    """
    cv = 0
    batch_size = cfg['batch_size']
    device = cfg['device']
    if isinstance(model, torch.nn.Module):
        batched_data, _ = test_sample

        n = batch_size
        m = math.floor(0.8 * n)

        background = batched_data[:m].to(device)
        test_data = batched_data[m:n].to(device)

        e = shap.DeepExplainer(model, background)
        shap_values = e.shap_values(test_data)
        if shap_values is not None and len(shap_values) > 0:
            sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
            abs_sums = np.absolute(sums)
            cv = variation(abs_sums)
    return cv